In [1]:
from pyspark.sql import SparkSession, functions as F
import os

spark_version = '3.4.1'
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:{spark_version},org.apache.spark:spark-sql-kafka-0-10_2.12:{spark_version} pyspark-shell'

import findspark
findspark.init(os.environ['SPARK_HOME'] )

In [2]:
spark = SparkSession.builder.appName('APP').config('hive.exec.dynamic.partition.mode', 'nonstrict').enableHiveSupport().getOrCreate()

In [3]:
spark

In [4]:
spark.sparkContext.setCheckpointDir('chkpoint')

In [5]:
df = spark.read.option("header", True).csv("datasets/Pokemon_full.csv")

In [28]:
dfc = df.unionByName(df.withColumn('height', F.col('height') + 1)).checkpoint()

In [7]:
dfc.show()

+----------+----------+------+------+------+--------------+---+------+-------+------+------+-----+
|      name|pokedex id|height|weight|  type|secundary type| hp|attack|defense|sp atk|sp def|speed|
+----------+----------+------+------+------+--------------+---+------+-------+------+------+-----+
| bulbasaur|         1|     7|    69| grass|        poison| 45|    49|     49|    65|    65|   45|
|   ivysaur|         2|    10|   130| grass|        poison| 60|    62|     63|    80|    80|   60|
|  venusaur|         3|    20|  1000| grass|        poison| 80|    82|     83|   100|   100|   80|
|charmander|         4|     6|    85|  fire|          None| 39|    52|     43|    60|    50|   65|
|charmeleon|         5|    11|   190|  fire|          None| 58|    64|     58|    80|    65|   80|
| charizard|         6|    17|   905|  fire|        flying| 78|    84|     78|   109|    85|  100|
|  squirtle|         7|     5|    90| water|          None| 44|    48|     65|    50|    64|   43|
| wartortl

In [30]:
dfl = df.unionByName(df.withColumn('height', F.col('height') + 1)).localCheckpoint()

In [10]:
dfl.toDF()

<bound method DataFrame.toDF of DataFrame[name: string, pokedex id: string, height: string, weight: string, type: string, secundary type: string, hp: string, attack: string, defense: string, sp atk: string, sp def: string, speed: string]>

## Clear filesystem

In [29]:
fs = (spark._jvm.org
      .apache.hadoop
      .fs.FileSystem
      .get(spark._jsc.hadoopConfiguration())
      )

srcPath = spark._jvm.org.apache.hadoop.fs.Path("chkpoint")

if fs.exists(srcPath):
    fs.delete(srcPath,True)